In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [5]:
# !pip install html_table_parser

In [6]:
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
import os

In [7]:
# # 날짜 dataframe 만들기
# start_date = pd.to_datetime('2015-01-01') ## 시작 날짜
# end_date = pd.to_datetime('2022-03-07') ## 마지막 날짜
 
# dates = pd.date_range(start_date,end_date,freq='M') ## 월단위로 생성
# dates = pd.DataFrame(dates, columns=['date'])

In [8]:
# 년,월,일 -> 날짜
def change_date(x):
    ret = []
    for da in x.split():
        ret.append(da[:-1])
    return '.'.join(ret)

In [9]:
# K, M, B 변환
def m_to_k(x):
    if x[-1] == 'M':
        return float(x[:-1]) * 1000
    elif x[-1] == 'K':
        return float(x[:-1])
    elif x[-1] == 'B':
        return float(x[:-1]) * 1000000
    else:
        return np.nan

In [10]:
# 나스닥 종합지수, 상해 종합지수, 달러지수 병합

In [13]:
pd.read_csv('./Data/나스닥종합지수.csv')

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2022년 03월 04일,"13,313.44","13,455.23","13,486.71","13,224.98",1.04B,-1.66%
1,2022년 03월 03일,"13,537.94","13,837.59","13,837.59","13,472.64",1.03B,-1.56%
2,2022년 03월 02일,"13,752.02","13,597.53","13,796.55","13,493.91",1.03B,1.62%
3,2022년 03월 01일,"13,532.46","13,716.70","13,777.04","13,441.32",1.24B,-1.59%
4,2022년 02월 28일,"13,751.40","13,570.83","13,810.64","13,549.90",1.39B,0.41%
...,...,...,...,...,...,...,...
1801,2015년 01월 08일,"4,736.19","4,689.54","4,741.38","4,688.02",548.10M,1.84%
1802,2015년 01월 07일,"4,650.47","4,626.84","4,652.72","4,613.90",482.51M,1.26%
1803,2015년 01월 06일,"4,592.74","4,666.85","4,667.33","4,567.60",553.62M,-1.29%
1804,2015년 01월 05일,"4,652.57","4,700.34","4,702.77","4,641.46",476.45M,-1.57%


In [20]:
## 나스닥
df_nasdaq = pd.read_csv('./Data/나스닥종합지수.csv')
df_nasdaq.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_nasdaq.rename(columns={'날짜':'date', '종가':'nasdaq', '거래량':'nasdaq_거래량', '변동 %':'nasdaq_volatility'},
                 inplace=True)

### 날짜 전처리
df_nasdaq.date = df_nasdaq.date.apply(change_date)
df_nasdaq.date = pd.to_datetime(df_nasdaq.date)

### 가격 float타입
df_nasdaq.nasdaq = df_nasdaq.nasdaq.str.replace(',','').astype('float')

### 거래량 전처리
df_nasdaq['nasdaq_거래량'] = df_nasdaq['nasdaq_거래량'].apply(m_to_k)

### 변동% 전처리
df_nasdaq['nasdaq_volatility'] = df_nasdaq.nasdaq_volatility.apply(lambda x : float(x[:-1]))

In [21]:
df_nasdaq

,date,nasdaq,nasdaq_거래량,nasdaq_volatility
0,2022-03-04,13313.44,1040000.0,-1.66
1,2022-03-03,13537.94,1030000.0,-1.56
2,2022-03-02,13752.02,1030000.0,1.62
3,2022-03-01,13532.46,1240000.0,-1.59
4,2022-02-28,13751.40,1390000.0,0.41
...,...,...,...,...
1801,2015-01-08,4736.19,548100.0,1.84
1802,2015-01-07,4650.47,482510.0,1.26
1803,2015-01-06,4592.74,553620.0,-1.29
1804,2015-01-05,4652.57,476450.0,-1.57


In [11]:
## 상해 종합지수
df_ssec = pd.read_csv('상하이종합 내역.csv')
df_ssec.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_ssec.rename(columns={'날짜':'date', '종가':'ssec', '거래량':'ssec_거래량', '변동 %':'ssec_volatility'},
               inplace=True)

### 날짜 전처리
df_ssec.date = df_ssec.date.apply(change_date)
df_ssec.date = pd.to_datetime(df_ssec.date)

### 가격 float타입
df_ssec.ssec = df_ssec.ssec.str.replace(',','').astype('float')

### 거래량 전처리
df_ssec['ssec_거래량'] = df_ssec['ssec_거래량'].apply(m_to_k)

### 변동% 전처리
df_ssec['ssec_volatility'] = df_ssec.ssec_volatility.apply(lambda x : float(x[:-1]))

In [12]:
## 달러지수
df_dollar = pd.read_csv('달러 지수.csv')
df_dollar.drop(['오픈','고가','저가', '거래량'], axis=1, inplace=True)
df_dollar.rename(columns={'날짜':'date', '종가':'dollar', '변동 %':'dollar_volatility'}, inplace=True)

### 날짜 전처리
df_dollar.date = df_dollar.date.apply(change_date)
df_dollar.date = pd.to_datetime(df_dollar.date)

### 변동% 전처리
df_dollar['dollar_volatility'] = df_dollar.dollar_volatility.apply(lambda x : float(x[:-1]))

In [13]:
## 병합
df_composite_index = pd.merge(df_nasdaq, df_ssec, how='outer', on='date')
df_composite_index = pd.merge(df_composite_index, df_dollar, how='outer', on='date')
# df_composite_index.to_csv('종합지수_merge.csv')

In [14]:
# 중국 2차전지, 전기차

In [15]:
## EVE Energy
df_eve = pd.read_csv('EVE Energy 주가.csv')
df_eve.drop(['오픈', '고가', '저가'], axis = 1, inplace = True)
df_eve.columns = 'date', 'EVE종가', 'EVE거래량', 'EVE변동률'
df_eve['date'] = pd.to_datetime(df_eve['date'].apply(change_date))
df_eve['EVE거래량'] = df_eve['EVE거래량'].apply(m_to_k).astype('float')
df_eve['EVE변동률'] = df_eve['EVE변동률'].apply(lambda x : x[:-1]).astype('float')

In [16]:
## BYD 주가
df_byd = pd.read_csv('BYD 주가.csv')
df_byd.drop(['오픈', '고가', '저가'], axis = 1, inplace=True)
df_byd.columns = 'date', 'BYD종가', 'BYD거래량', 'BYD변동률'
df_byd['BYD거래량'] = df_byd['BYD거래량'].apply(m_to_k).astype('float')
df_byd['date'] = pd.to_datetime(df_byd['date'].apply(change_date))
df_byd['BYD변동률'] = df_byd['BYD변동률'].apply(lambda x : x[:-1]).astype('float')

In [17]:
## eve, byd merge
merge_eve_byd = pd.merge(df_eve, df_byd, on = 'date', how = 'outer')
# merge_eve_byd.to_csv('중국2차전지.csv', header = False, index = False)

In [18]:
# 희토류, 리튬, 코발트 병합

In [19]:
## 리튬
df_li = pd.read_excel('리튬+가격2022-03-08.xls')
df_li.drop(['등락가', '등락비율', '재고량'], axis=1, inplace=True)
df_li.columns = ['date', 'li_price']
df_li.date = pd.to_datetime(df_li.date)
# df_li.to_csv('data_li.csv')

df_li = pd.read_csv('data_li.csv')
df_li.date = pd.to_datetime(df_li.date)

In [20]:
## 코발트
df_co = pd.read_csv('코발트 가격.csv')
df_co.drop(['오픈','고가','저가','거래량'], axis=1, inplace=True)
df_co.rename(columns={'날짜':'date', '종가':'co', '변동 %':'co_volatility'},
                 inplace=True)

### 날짜 전처리
df_co.date = df_co.date.apply(change_date)
df_co.date = pd.to_datetime(df_co.date)

### 가격 float타입
df_co.co = df_co.co.str.replace(',','').astype('float')

### 변동% 전처리
df_co['co_volatility'] = df_co.co_volatility.apply(lambda x : float(x[:-1]))

In [21]:
## 희토류
df_ra = pd.read_excel('희토류 가격.xlsx')
df_ra.drop(['등락가', '재고량', '등락비율'], axis = 1, inplace = True)
df_ra.columns = 'date', '희토류가격'
df_ra.date = pd.to_datetime(df_ra.date)

In [22]:
## 병합
merge_lico = pd.merge(df_li, df_co, on = 'date', how = 'outer')
merge_licora = pd.merge(merge_lico, df_ra, on = 'date', how = 'outer')
# merge_licora.to_csv('배터리원자재_merge.csv')

In [23]:
# 재무제표

In [14]:
## 재무제표1 크롤링
url = 'https://www.choicestock.co.kr/search/invest/TSLA/MRQ'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
datas = soup.select('table.tableRanking')

## 데이터 프레임 생성
li = []
for data in datas:
    table = parser_functions.make2d(data)
    df = pd.DataFrame(data=table[1:], columns=table[0])
    li.append(df)
    
## 합병
df = pd.concat(li)

## 저장
# df.to_csv('fn_statement.csv')

In [15]:
df

,,21.12/31,21.09/30,21.06/30,21.03/31,20.12/31,20.09/30,20.06/30,20.03/31,19.12/31,...,13.09/30,13.06/30,13.03/31,12.12/31,12.09/30,12.06/30,12.03/31,초이스스탁US 프리미엄,프리미엄,무료
0,주당순이익EPS(달러),2.06,1.44,1.02,0.39,0.24,0.27,0.10,0.02,0.13,...,-0.06,-0.05,0.00,-0.16,-0.21,-0.20,-0.17,NaN,NaN,NaN
1,주당순자산BPS(달러),29.80,27.11,25.55,23.95,23.37,17.11,10.62,10.03,7.34,...,0.93,1.07,0.29,0.22,-0.05,0.12,0.29,NaN,NaN,NaN
2,주당배당금DPS(달러),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
3,주가수익배수PER(배),192.30,221.38,300.22,560.91,927.75,718.98,543.93,-659.88,-87.47,...,-159.02,-56.39,-14.70,-9.73,-7.96,-9.63,-13.19,NaN,NaN,NaN
4,주가순자산배수PBR(배),35.16,28.38,26.40,27.85,30.10,24.94,20.31,10.36,11.39,...,41.63,19.71,25.74,30.90,N/A,52.92,25.32,NaN,NaN,NaN
5,주가매출액배수PSR(배),19.72,16.39,15.64,17.84,21.21,14.19,7.79,3.65,3.07,...,13.78,9.37,4.59,9.33,21.10,21.40,21.01,NaN,NaN,NaN
6,자기자본이익률ROE(%),N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,NaN,NaN,NaN
7,총자산이익률ROA(%),N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,NaN,NaN,NaN
8,투하자본이익률ROIC(%),N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,NaN,NaN,NaN
9,부채비율(%),101.20,108.50,116.50,123.90,127.90,175.60,271.50,289.10,395.90,...,270.20,199.90,578.50,793.50,N/A,"1,148.70",394.60,NaN,NaN,NaN


In [19]:
pd.read_csv('./Data/tesla 재무제표.csv')

,Unnamed: 0,Unnamed: 1,21.12/31,21.09/30,21.06/30,21.03/31,20.12/31,20.09/30,20.06/30,20.03/31,...,13.09/30,13.06/30,13.03/31,12.12/31,12.09/30,12.06/30,12.03/31,초이스스탁US 프리미엄,프리미엄,무료
0,0,매출액,"53,823","46,848","41,862","35,940","31,536","28,176","25,708","26,022",...,"1,705","1,323",945,413,146.0,154.00,185.0,NaN,NaN,NaN
1,1,매출원가,"40,217","36,023","32,634","28,329","24,906","22,221","20,625","21,285",...,"1,381","1,111",829,383,132.0,114.00,132.0,NaN,NaN,NaN
2,2,매출총이익,"13,606","10,825","9,228","7,611","6,630","5,955","5,083","4,737",...,324,212,116,30,14.0,40.00,54.0,NaN,NaN,NaN
3,3,판매관리비,"4,517","3,992","3,886","3,574","3,145","2,875","2,583","2,569",...,230,191,167,150,132.0,122.00,110.0,NaN,NaN,NaN
4,4,연구개발비,"2,593","2,375","2,130","1,833","1,491","1,314","1,282","1,327",...,232,238,260,274,266.0,259.00,236.0,NaN,NaN,NaN
5,5,영업이익,"6,523","4,485","3,290","2,305","1,994","1,778","1,230",736,...,-139,-217,-311,-394,-384.0,-340.00,-293.0,NaN,NaN,NaN
6,6,EBITDA,"9,500","7,185","5,721","4,570","4,083","3,760","3,380","2,880",...,-39,-139,-252,-367,-366.0,-324.00,-277.0,NaN,NaN,NaN
7,7,법인세,699,490,453,359,292,251,91,89,...,1.08,0.42,0.23,0.14,0.4,0.37,0.4,NaN,NaN,NaN
8,8,중단손익,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.00,0.0,NaN,NaN,NaN
9,9,순이익,"5,519","3,468","2,181","1,143",721,556,368,-144,...,-148,-220,-295,-396,-388.0,-342.00,-295.0,NaN,NaN,NaN


In [17]:
## 재무제표1 전처리
df_fn_statements = pd.read_csv('./Data/tesla 재무제표.csv')
df_fn_statements = df_fn_statements.set_index('Unnamed: 1')
df_fn_statements = df_fn_statements.drop(['Unnamed: 0','초이스스탁US 프리미엄', '프리미엄', '무료'], axis = 1)
df_fn_statements.dropna(inplace = True)
df_fn_statements = df_fn_statements.T
df_fn_statements = df_fn_statements.astype('object')
df_fn_statements = df_fn_statements.reset_index()
df_fn_statements = df_fn_statements.rename_axis(columns={'Unnamed: 1' : ''})
df_fn_statements = df_fn_statements.rename(columns={'index':'date'})
df_fn_statements.date = df_fn_statements.date.apply(lambda x : x.replace('.','/'))
df_fn_statements.date = df_fn_statements.date.apply(lambda x : '/'.join(x.split('/')[::-1]))
df_fn_statements.date = pd.to_datetime(df_fn_statements.date)
df_fn_statements.drop(['중단손익'], axis=1, inplace=True)

drop_col = ['매입채무등', '단기차입금', '장기차입금', '이연수익', '이연세부채','기타부채', '부채총계',
            '이익잉여금', '기타포괄익', '자본총계', '영업활동', '투자활동', '재무활동', '환율변동',
            '현금의증감', '자본적지출', '잉여현금', '현금성자산', '단기투자', '매출채권등', '재고자산',
            '장기투자', '유형자산', '무형자산', '이연세자산', '기타자산','EBITDA', '법인세', '판매관리비',
            '연구개발비']

df_fn_statements.drop(drop_col, axis=1, inplace=True)
df_fn_statements = df_fn_statements.iloc[:28]

## float 타입변환
col = df_fn_statements.columns.values
for c in col[1:]:
    df_fn_statements.loc[:,c] = df_fn_statements.loc[:,c].apply(lambda x : x.replace(',',''))
    df_fn_statements[c] = df_fn_statements[c].astype('float')

## 이익률 계산
df_fn_statements['매출총이익률'] = (df_fn_statements['매출총이익']/df_fn_statements['매출액'])*100
df_fn_statements['영업이익률'] = (df_fn_statements['영업이익']/df_fn_statements['매출액'])*100
# df_fn_statements.to_csv('fn_statements.csv')

In [18]:
df_fn_statements

,date,매출액,매출원가,매출총이익,영업이익,순이익,자산총계,매출총이익률,영업이익률
0,2021-12-31,53823.0,40217.0,13606.0,6523.0,5519.0,62131.0,25.279156,12.119354
1,2021-09-30,46848.0,36023.0,10825.0,4485.0,3468.0,57834.0,23.106643,9.573514
2,2021-06-30,41862.0,32634.0,9228.0,3290.0,2181.0,55146.0,22.043858,7.859156
3,2021-03-31,35940.0,28329.0,7611.0,2305.0,1143.0,52972.0,21.176962,6.413467
4,2020-12-31,31536.0,24906.0,6630.0,1994.0,721.0,52148.0,21.023592,6.322933
5,2020-09-30,28176.0,22221.0,5955.0,1778.0,556.0,45691.0,21.135009,6.310335
6,2020-06-30,25708.0,20625.0,5083.0,1230.0,368.0,38135.0,19.772055,4.784503
7,2020-03-31,26022.0,21285.0,4737.0,736.0,-144.0,37250.0,18.203828,2.828376
8,2019-12-31,24578.0,20509.0,4069.0,-69.0,-862.0,34309.0,16.555456,-0.280739
9,2019-09-30,24420.0,20299.0,4121.0,-14.0,-827.0,32795.0,16.875512,-0.057330


In [22]:
## 재무제표1 보간법
### 날짜 dataframe 만들기
start_date = pd.to_datetime('2015-01-01') ## 시작 날짜
end_date = pd.to_datetime('2022-03-07') ## 마지막 날짜
 
dates = pd.date_range(start_date,end_date,freq='D') ## 일단위로 생성
dates = pd.DataFrame(dates, columns=['date'])
merge_fn_statement1_date = pd.merge(dates, df_fn_statements, how='left', on='date')
## 보간법
col = merge_fn_statement1_date.columns.values
for c in col[1:]:
    merge_fn_statement1_date.loc[:,c] = merge_fn_statement1_date.loc[:,c].interpolate()

In [23]:
merge_fn_statement1_date

,date,매출액,매출원가,매출총이익,영업이익,순이익,자산총계,매출총이익률,영업이익률
0,2015-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2618,2022-03-03,53823.0,40217.0,13606.0,6523.0,5519.0,62131.0,25.279156,12.119354
2619,2022-03-04,53823.0,40217.0,13606.0,6523.0,5519.0,62131.0,25.279156,12.119354
2620,2022-03-05,53823.0,40217.0,13606.0,6523.0,5519.0,62131.0,25.279156,12.119354
2621,2022-03-06,53823.0,40217.0,13606.0,6523.0,5519.0,62131.0,25.279156,12.119354


In [25]:
## 재무제표2 크롤링
url = 'https://www.choicestock.co.kr/search/invest/TSLA'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
datas = soup.select('table')
li = []
for data in datas:
    table = parser_functions.make2d(data)
    df = pd.DataFrame(data=table[1:], columns=table[0])
    li.append(df)
### 병합
df = pd.concat(li)

In [26]:
## 재무제표2 전처리

In [27]:
df = df.iloc[:12, :].drop(['초이스스탁US 프리미엄', '프리미엄', '무료'], axis=1).set_index('').T
df.to_csv('재무재표2_전처리.csv')

In [28]:
df_fn_statements2 = pd.read_csv('재무재표2_전처리.csv')
df_fn_statements2 = df_fn_statements2.rename(columns={'Unnamed: 0':'date'})
df_fn_statements2.date = df_fn_statements2.date.apply(lambda x : x.replace('.','/'))
df_fn_statements2.date = df_fn_statements2.date.apply(lambda x : '/'.join(x.split('/')[::-1]))
df_fn_statements2.date = pd.to_datetime(df_fn_statements2.date)
df_fn_statements2.drop(['부채비율(%)','이자보상배수(배)','주당순자산BPS(달러)','주당배당금DPS(달러)','주가순자산배수PBR(배)',
                        '주가매출액배수PSR(배)','자기자본이익률ROE(%)','투하자본이익률ROIC(%)','유동비율(%)'], axis=1, inplace=True)

In [29]:
## 재무제표2 보간법
### 날짜 dataframe 만들기
start_date = pd.to_datetime('2015-01-01') ## 시작 날짜
end_date = pd.to_datetime('2022-03-07') ## 마지막 날짜
 
dates = pd.date_range(start_date,end_date,freq='D') ## 일단위로 생성
dates = pd.DataFrame(dates, columns=['date'])
merge_fn_statement2_date = pd.merge(dates, df_fn_statements2, how='left', on='date')
## 보간법
col = merge_fn_statement2_date.columns.values
for c in col[1:]:
    merge_fn_statement2_date.loc[:,c] = merge_fn_statement2_date.loc[:,c].interpolate()

In [30]:
## 재무제표 merge
merge_fn_statement = pd.merge(merge_fn_statement1_date, merge_fn_statement2_date, how='outer', on='date')
# merge_fn_statement.to_csv('재무제표 merge.csv')

In [31]:
merge_fn_statement

,date,매출액,매출원가,매출총이익,영업이익,순이익,자산총계,매출총이익률,영업이익률,주당순이익EPS(달러),주가수익배수PER(배),총자산이익률ROA(%)
0,2015-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2618,2022-03-03,53823.0,40217.0,13606.0,6523.0,5519.0,62131.0,25.279156,12.119354,4.9,192.3,9.7
2619,2022-03-04,53823.0,40217.0,13606.0,6523.0,5519.0,62131.0,25.279156,12.119354,4.9,192.3,9.7
2620,2022-03-05,53823.0,40217.0,13606.0,6523.0,5519.0,62131.0,25.279156,12.119354,4.9,192.3,9.7
2621,2022-03-06,53823.0,40217.0,13606.0,6523.0,5519.0,62131.0,25.279156,12.119354,4.9,192.3,9.7


In [33]:
# 리튬관련 etf

In [34]:
## LIT etf
df_lit = pd.read_csv('리튬 etf LIT.csv')
df_lit.drop(['Open', 'High', 'Low', 'Close'], axis = 1, inplace=True)
df_lit.columns = 'date', 'LIT조정종가', 'LIT거래량'
df_lit['date'] = pd.to_datetime(df_lit['date'])

In [35]:
## SGL ETF
df_sgl = pd.read_csv('리튬전지 SGL etf.csv')
df_sgl.drop(['오픈', '고가', '저가', '거래량'], axis = 1, inplace = True)
df_sgl.columns = 'date', 'SGL종가', 'SGL변동률'
df_sgl['date'] = df_sgl['date'].apply(change_date)
df_sgl['date'] = pd.to_datetime(df_sgl['date'])
### 변동% 전처리
df_sgl['SGL변동률'] = df_sgl.SGL변동률.apply(lambda x : float(x[:-1]))

In [36]:
## 리튬 etf 병합
df_lithium_etf = pd.merge(df_lit, df_sgl, on = 'date', how = 'outer')
# df_lithium_etf.to_csv('lithium_etf_merge')

In [37]:
# 반도체관련 etf

In [38]:
## psi etf (미국)
df_psi = pd.read_csv("미국 반도체 etf PSI.csv")
df_psi = df_psi.rename(columns={'날짜':'date'})
df_psi = df_psi.rename(columns={'거래량':'PSI거래량','종가':'PSI종가'})
df_psi.drop(['오픈','고가','저가','변동 %'], axis=1, inplace=True)
df_psi.date = pd.to_datetime(df_psi.date.apply(change_date))
df_psi.PSI거래량 = df_psi.PSI거래량.apply(m_to_k).astype('float')

In [39]:
## soxx etf (미국)
df_soxx = pd.read_csv("미국 반도체 etf SOXX.csv")
df_soxx = df_soxx.rename(columns = {'날짜' : 'date','거래량' : 'SOXX거래량','종가' : 'SOXX종가'})
df_soxx.drop(['오픈','고가','저가','변동 %'], axis=1, inplace=True)
df_soxx.date = pd.to_datetime(df_soxx.date.apply(change_date))
df_soxx.SOXX거래량 = df_soxx.SOXX거래량.apply(m_to_k).astype("float")

In [40]:
## soxl etf (미국)
df_soxl = pd.read_csv("미국 반도체 etf SOXL.csv")
df_soxl = df_soxl.rename(columns = {'날짜' : 'date','거래량' : 'SOXL거래량','종가' : 'SOXL종가'})
df_soxl.drop(['오픈','고가','저가','변동 %'], axis=1, inplace=True)
df_soxl.date = pd.to_datetime(df_soxl.date.apply(change_date))
df_soxl.SOXL거래량 = df_soxl.SOXL거래량.apply(m_to_k).astype("float")

In [41]:
## 반도체 etf merge
merge_outer = pd.merge(df_psi, df_soxx,how='outer', on='date')
semi_etf_merge = pd.merge(merge_outer,df_soxl, how='outer', on='date')
# semi_etf_merge.to_csv('반도체 etf merge.csv')

In [42]:
# 전기차관련 etf

In [43]:
## driv etf (미국)
df_driv = pd.read_csv("미국 전기차 erf DRIV.csv")
df_driv = df_driv.rename(columns = {'날짜':'date', '종가':'DRIV종가','거래량':'DRIV거래량'})
df_driv.drop(['오픈','고가','저가','변동 %'], axis =1, inplace=True)
df_driv.date = pd.to_datetime(df_driv.date.apply(change_date))
df_driv.DRIV거래량 = df_driv.DRIV거래량.apply(m_to_k).astype('float')

In [44]:
## acdc etf (캐나다)
df_acdc = pd.read_csv("캐나다 전기차 etf ACDC.csv")
df_acdc = df_acdc.rename(columns = {'날짜':'date', '종가':'ACDC종가','거래량':'ACDC거래량'})
df_acdc.drop(['오픈','고가','저가','변동 %'], axis =1, inplace=True)
df_acdc.date = pd.to_datetime(df_acdc.date.apply(change_date))
df_acdc.ACDC거래량 = df_acdc.ACDC거래량.apply(m_to_k).astype('float')

In [45]:
## kars (미국 자율주행 etf)
df_kars = pd.read_csv("미국 자율주행전기차 etf KARS.csv")
df_kars.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_kars.rename(columns={'날짜':'date', '종가':'kars', '거래량':'kars_거래량', '변동 %':'kars_volatility'},
                 inplace=True)

### 날짜 전처리
df_kars.date = df_kars.date.apply(change_date)
df_kars.date = pd.to_datetime(df_kars.date)

### 가격 float타입
df_kars.kars = df_kars.kars.astype('float')

### 거래량 전처리
df_kars['kars_거래량'] = df_kars['kars_거래량'].apply(m_to_k)

### 변동% 전처리
df_kars['kars_volatility'] = df_kars.kars_volatility.apply(lambda x : float(x[:-1]))

In [46]:
## BATT (미국 배터리 etf)
df_batt = pd.read_csv("미국 배터리 etf BATT.csv")
df_batt.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_batt.rename(columns={'날짜':'date', '종가':'batt', '거래량':'batt_거래량', '변동 %':'batt_volatility'},
                 inplace=True)

### 날짜 전처리
df_batt.date = df_batt.date.apply(change_date)
df_batt.date = pd.to_datetime(df_batt.date)

### 가격 float타입
df_batt.batt = df_batt.batt.astype('float')

### 거래량 전처리
df_batt['batt_거래량'] = df_batt['batt_거래량'].apply(m_to_k)

### 변동% 전처리
df_batt['batt_volatility'] = df_batt.batt_volatility.apply(lambda x : float(x[:-1]))

In [47]:
## 전기차 etf 병합
ele_car_merge = pd.merge(df_driv, df_acdc, how='outer', on='date')
ele_car_merge = pd.merge(ele_car_merge, df_kars, how='outer', on='date')
ele_car_merge = pd.merge(ele_car_merge, df_batt, how='outer', on='date')
# ele_car_merge.to_csv('전기차 etf merge.csv')

In [48]:
## TBX (미국채금리 인버스 etf)

In [49]:
df_tbx = pd.read_csv("미국채금리 인버스 etf TBX.csv")
df_tbx.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_tbx.rename(columns={'날짜':'date', '종가':'tbx', '거래량':'tbx_거래량', '변동 %':'tbx_volatility'},
                 inplace=True)

### 날짜 전처리
df_tbx.date = df_tbx.date.apply(change_date)
df_tbx.date = pd.to_datetime(df_tbx.date)

### 가격 float타입
df_tbx.tbx = df_tbx.tbx.astype('float')

### 거래량 전처리
df_tbx['tbx_거래량'] = df_tbx['tbx_거래량'].apply(m_to_k)

### 변동% 전처리
df_tbx['tbx_volatility'] = df_tbx.tbx_volatility.apply(lambda x : float(x[:-1]))

In [50]:
# 한국 2차전지 etf

In [51]:
## kodex
df_kodex = pd.read_csv('한국 2차전지 etf KODEX 2차전지산업.csv')
df_kodex.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_kodex.rename(columns={'날짜':'date', '종가':'kodex', '거래량':'kodex_거래량', '변동 %':'kodex_volatility'},
                 inplace=True)

### 날짜 전처리
df_kodex.date = df_kodex.date.apply(change_date)
df_kodex.date = pd.to_datetime(df_kodex.date)

### 가격 float타입
df_kodex.kodex = df_kodex.kodex.str.replace(',','').astype('float')

### 거래량 전처리
df_kodex['kodex_거래량'] = df_kodex['kodex_거래량'].apply(m_to_k)

### 변동% 전처리
df_kodex['kodex_volatility'] = df_kodex.kodex_volatility.apply(lambda x : float(x[:-1]))

In [52]:
## tiger
df_tiger = pd.read_csv('타이거 2차전지 etf.csv')
df_tiger.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_tiger.rename(columns={'날짜':'date', '종가':'tiger', '거래량':'tiger_거래량', '변동 %':'tiger_volatility'},
                 inplace=True)

### 날짜 전처리
df_tiger.date = df_tiger.date.apply(change_date)
df_tiger.date = pd.to_datetime(df_tiger.date)

### 가격 float타입
df_tiger.tiger = df_tiger.tiger.str.replace(',','').astype('float')

### 거래량 전처리
df_tiger['tiger_거래량'] = df_tiger['tiger_거래량'].apply(m_to_k)

### 변동% 전처리
df_tiger['tiger_volatility'] = df_tiger.tiger_volatility.apply(lambda x : float(x[:-1]))

In [53]:
btr_merge = pd.merge(df_kodex, df_tiger, how='outer', on='date')

In [54]:
# 테슬라 주가(target)

In [55]:
## 테슬라
df_tesla = pd.read_csv('테슬라 주가.csv')
df_tesla.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_tesla.rename(columns={'날짜':'date', '종가':'tesla', '거래량':'tesla_거래량', '변동 %':'tesla_volatility'},
                 inplace=True)

### 가격 float타입
df_tesla.tesla = df_tesla.tesla.str.replace(',','').astype('float')

### 날짜 전처리
df_tesla.date = df_tesla.date.apply(change_date)
df_tesla.date = pd.to_datetime(df_tesla.date)

### 거래량 전처리
df_tesla['tesla_거래량'] = df_tesla['tesla_거래량'].apply(m_to_k)

### 변동% 전처리
df_tesla['tesla_volatility'] = df_tesla.tesla_volatility.apply(lambda x : float(x[:-1]))

In [56]:
# 유가 선물지수, 탄소배출권

In [57]:
## 유가 선물지수
df_oil_price = pd.read_csv('국제 유가 선물.csv')
df_oil_price.drop(['오픈','고가','저가', '거래량'], axis=1, inplace=True)
df_oil_price.rename(columns={'날짜':'date', '종가':'oil_price', '변동 %':'oil_price_volatility'}, inplace=True)

### 날짜 전처리
df_oil_price.date = df_oil_price.date.apply(change_date)
df_oil_price.date = pd.to_datetime(df_oil_price.date)

### 변동% 전처리
df_oil_price['oil_price_volatility'] = df_oil_price.oil_price_volatility.apply(lambda x : float(x[:-1]))

In [58]:
## 탄소배출권
df_carbon_credits = pd.read_csv('탄소배출권.csv')
df_carbon_credits.drop(['오픈','고가','저가', '거래량'], axis=1, inplace=True)
df_carbon_credits.rename(columns={'날짜':'date', '종가':'carbon_credits', '변동 %':'carbon_credits_volatility'}, inplace=True)

### 날짜 전처리
df_carbon_credits.date = df_carbon_credits.date.apply(change_date)
df_carbon_credits.date = pd.to_datetime(df_carbon_credits.date)

### 변동% 전처리
df_carbon_credits['carbon_credits_volatility'] = df_carbon_credits.carbon_credits_volatility.apply(lambda x : float(x[:-1]))

In [59]:
## 유가, 탄소배출권 병합
merge_oil_carbon = pd.merge(df_oil_price, df_carbon_credits, how='outer', on='date')
# merge_oil_carbon.to_csv('유가 탄소 merge.csv')

In [60]:
# 미국 전기세

In [61]:
## 미국 전기세 전처리
df_e_fee_usa = pd.read_csv('미국 전기세.csv', skiprows=4)
df_e_fee_usa = df_e_fee_usa.drop(['units','source key'], axis=1)
df_e_fee_usa = df_e_fee_usa.set_index('description').T
df_e_fee_usa = pd.DataFrame(df_e_fee_usa['United States : industrial'])
df_e_fee_usa = df_e_fee_usa.reset_index()
df_e_fee_usa = df_e_fee_usa.rename(columns={'United States : industrial' : 'elec_fee', 'index':'date'})
df_e_fee_usa = df_e_fee_usa.iloc[168:]
df_e_fee_usa.date = pd.to_datetime(df_e_fee_usa['date'])
df_e_fee_usa.elec_fee = df_e_fee_usa.elec_fee.astype('float')

In [62]:
# 알류미늄, 구리, 신재생

In [63]:
## REGI(신재생)
### REGI 데이터 전처리
df_regi = pd.read_csv('./REGI 역사적 데이터/REGI 역사적 데이터.csv')
df_regi.drop(['오픈', '고가', '저가'], axis=1, inplace=True)
df_regi.columns = ['date', 'REGI_price', 'REGI_vol', 'REGI_change']
df_regi.date = df_regi.date.apply(change_date)
df_regi.date = pd.to_datetime(df_regi.date)
df_regi.REGI_vol = df_regi.REGI_vol.apply(m_to_k)
df_regi['REGI_change'] = df_regi['REGI_change'].apply(lambda x: float(x.replace('%', '')))

In [64]:
## 알루미늄 전처리
df_alu = pd.read_csv('알루미늄 내역.csv')
df_alu.drop(['오픈', '고가', '저가'], axis=1, inplace=True)
df_alu.columns = ['date', 'alu_price', 'alu_vol', 'alu_change']
df_alu.date = df_alu.date.apply(change_date)
df_alu.date = pd.to_datetime(df_alu.date)
df_alu['alu_price'] = df_alu['alu_price'].apply(lambda x: x.replace(',', ''))
df_alu['alu_price'] = pd.to_numeric(df_alu['alu_price'])
df_alu['alu_vol'] = df_alu['alu_vol'].apply(m_to_k)
df_alu['alu_change'] = df_alu['alu_change'].apply(lambda x: float(x.replace('%', '')))

In [65]:
## 구리 데이터 전처리
df_cu = pd.read_csv('영국 구리 선물 내역.csv')
df_cu.drop(['오픈', '고가', '저가', '거래량'], axis=1, inplace=True)
df_cu.columns = ['date', 'cu_price', 'cu_change']
df_cu.date = df_cu.date.apply(change_date)
df_cu.date = pd.to_datetime(df_cu.date)
df_cu.cu_price = df_cu.cu_price.apply(lambda x: float(x.replace(',', '')))
df_cu['cu_change'] = df_cu['cu_change'].apply(lambda x: float(x.replace('%', '')))

In [66]:
## 알류미늄, 구리, 신재생 병합
al_cu_regi_merge = pd.merge(df_alu, df_cu, on='date', how='outer')
al_cu_regi_merge = pd.merge(al_cu_regi_merge, df_regi, on='date', how='outer')
# al_cu_regi_merge = pd.read_csv('alu_cu_REGI_df.csv')
# al_cu_regi_merge.drop('Unnamed: 0', axis=1, inplace=True)
al_cu_regi_merge.date = pd.to_datetime(al_cu_regi_merge.date)

In [67]:
## S&P 500
df_snp_500 = pd.read_csv('S&P 500 내역.csv')
df_snp_500.drop(['오픈','고가','저가', '거래량'], axis=1, inplace=True)
df_snp_500.rename(columns={'날짜':'date', '종가':'snp_500', '거래량':'snp_500_거래량', '변동 %':'snp_500_volatility'},
               inplace=True)

### 날짜 전처리
df_snp_500.date = df_snp_500.date.apply(change_date)
df_snp_500.date = pd.to_datetime(df_snp_500.date)

### 가격 float타입
df_snp_500.snp_500 = df_snp_500.snp_500.str.replace(',','').astype('float')

### 변동% 전처리
df_snp_500['snp_500_volatility'] = df_snp_500.snp_500_volatility.apply(lambda x : float(x[:-1]))

In [68]:
## 검색량

In [69]:
datas = os.listdir('./multiTimeline')
li = []
for data in datas:
    df = pd.read_csv('./multiTimeline/{}'.format(data))
    li.append(df)
df_con = pd.concat(li)
df_con = df_con.reset_index()
df_con = df_con.rename(columns={'index' : 'date', '카테고리: 모든 카테고리' : '검색량'})
df_con.검색량 = df_con.검색량.apply(lambda x: x if x.isnumeric() else np.nan)
df_con.dropna(inplace=True)
df_con.date = pd.to_datetime(df_con.date)
df_con.to_csv('검색량.csv')

In [70]:
# 테슬라 주가 기준 merge

In [71]:
## 테슬라 - 미국전기세
df_total = pd.merge(df_tesla, df_e_fee_usa, how='outer', on='date')
df_total = df_total.sort_values('date')
### 전기세의 nan값은 ffill(한달 단위기 때문에)
df_total.elec_fee.fillna(method='ffill', inplace=True)
df_total.dropna(how='any',inplace=True)

In [72]:
## 테슬라 - 유가,탄소
df_total = pd.merge(df_total, merge_oil_carbon, how='left', on='date')

In [73]:
## 테슬라 - 중국 전기차 주가
df_total = pd.merge(df_total, merge_eve_byd, how='left', on='date')

In [74]:
# ## 테슬라 - 전기차 etf
df_total = pd.merge(df_total, ele_car_merge, how='left', on='date')

In [75]:
## 테슬라 - 반도체 etf
df_total = pd.merge(df_total, semi_etf_merge, how='left', on='date')

In [76]:
## 테슬라 - 리튬 etf
df_total = pd.merge(df_total, df_lithium_etf, how='left', on='date')

In [77]:
## 테슬라 - 희토류, 리튬, 코발트
df_total = pd.merge(df_total, merge_licora, how='left', on='date')

### 리튬 nan값 선형보간
df_total['li_price'] = df_total['li_price'].interpolate()

### 희토류 nan값 선형보간
df_total.희토류가격 = df_total.희토류가격.interpolate()

### 코발트 nan값 선형보간
df_total.co = df_total.co.interpolate()
df_total['co_volatility'] = df_total['co_volatility'].interpolate()

In [78]:
## 테슬라 - 종합지수
df_total = pd.merge(df_total, df_composite_index, how='left', on='date')

In [79]:
## 테슬라 - 알루미늄, 구리, 신재생
df_total = pd.merge(df_total, al_cu_regi_merge, how='left', on='date')

In [80]:
## 테슬라 - S&P 500
df_total = pd.merge(df_total, df_snp_500, how='left', on='date')

In [81]:
## 테슬라 - 2차전지
df_total = pd.merge(df_total, btr_merge, how='left', on='date')

In [82]:
## 테슬라 - tbx
df_total = pd.merge(df_total, df_tbx, how='left', on='date')

In [83]:
## 테슬라 - 검색량
df_total = pd.merge(df_total, df_con, how='left', on='date')

In [84]:
## 테슬라 - 재무제표
df_total = pd.merge(df_total, merge_fn_statement, how='left', on='date')

In [85]:
# ## 2차전지, 전기차 etf는 2019년부터 데이터가 있기 때문에 nan값을 따로 채우지 않고 2019년 이전의 데이터를 드랍
# ### 나머지 선형보간
# col_total = df_total.columns.values
# for col in col_total[1:]:
#     df_total[col] = df_total[col].interpolate()

# ### 나머지 nan값 처리
# df_total.fillna(method='bfill', inplace=True)

In [86]:
# # 2015년부터 있는 데이터만
# df_total.to_csv('테슬라 총합.csv')

In [87]:
# # 2019년부터 있는 데이터도 포함
# df_total.to_csv('테슬라 총합 2019.csv')

In [89]:
# # 19년부터 있는 데이터 + 재무제표
# df_total.to_csv('테슬라 총합 재무제표-1.csv', encoding='utf-8')